In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
#train_dataset = datasets.MNIST(root='./mnist_data/',
#                               train=True,
#                               transform=transforms.ToTensor(),
#                               download=True)
#
#test_dataset = datasets.MNIST(root='./mnist_data/',
#                              train=False,
#                              transform=transforms.ToTensor())

#Load as panda
pd_train_dataset = pd.read_csv('./kaggle_data/train.csv')
pd_test_dataset = pd.read_csv('./kaggle_data/test.csv')


In [3]:
class mnist_dataset_train(Dataset):
    """ train mnist dataset."""

    # Initialize your data, download, etc.
    def __init__(self):

        #Train data
        self.len = pd_train_dataset.shape[0]
        
        self.pt_x_data = torch.from_numpy(np.array(pd_train_dataset.iloc[:,1:].values, dtype='float')).float()
        self.y_data = np.array(pd_train_dataset.iloc[:, :1].values, dtype='float')
        self.pt_y_data = torch.from_numpy(self.y_data).long()
        
     
    def __getitem__(self, index):
        return self.pt_x_data[index], self.pt_y_data[index]

    def __len__(self):
        return self.len


train_dataset = mnist_dataset_train()

In [4]:
print (train_dataset.pt_y_data[0,:])
print (train_dataset.pt_y_data.shape)

tensor([1])
torch.Size([42000, 1])


In [5]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=0)

#test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                          batch_size=batch_size,
#                                          shuffle=False,
#                                          num_workers=0)

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

In [7]:
#criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
old
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        #print (output.shape)
        #print (target.shape)
        
        target = target.squeeze_()
        #print (target.shape)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [10]:
for epoch in range(1, 10):
    train(epoch)
    #test()

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [10, 1, 5, 5], but got input of size [64, 784] instead

In [15]:
#Test data
pt_x_data_test = torch.from_numpy(np.array(pd_test_dataset.iloc[:,:].values, dtype='float')).float()
#y_data_test = np.array(validation_set.loc[:, ['survived']].values, dtype='float')
#pt_y_data_test = torch.from_numpy(self.y_data_test).float()

In [31]:
#Run test set through
model.eval()
test_pred = model(pt_x_data_test)
#print (test_pred.shape)
#print (type(test_pred))
#print (test_pred[0,:])

pred = test_pred.data.max(1, keepdim=True)[1]
print (pred)

to_np = pred.detach().numpy()


#Add prediction col to test panda
#to_np = to_np.astype(int)
pd_test_dataset['Label'] = to_np


#Export to csv for submission

pd_test_dataset.index = np.arange(1, len(pd_test_dataset)+1)
pd_test_dataset.to_csv('test.csv', columns = ['Label'], index = True, index_label = 'ImageId')

tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]])


tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]])
